In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                # if file_from < date_obj:
                #     continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

# df_all = []
# for i in range(15):
    # Directory containing your event files
log_dir = f"/home/<User>/repos/csg_is/calculating_scores/classical_ml/version_9"
# date_str = "02/07/2024 10:22:20" # minus two hours for timezone

# date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

df = event_files_to_dataframe(log_dir)
print(df.head())

# df.sort_values(by="timestamp", inplace=True)

# files_t.sort(reverse=True)
# print(files_t[0], files_t[-1])

# df[df["tag"]=='Test/Loss']["timestamp"].max()

# test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
# print(test_df)
# test_df["participant"] = f"participant_{i}"
# df_all.append(test_df)

   step     value                     timestamp         tag
0    49  1.411568 2024-07-19 10:09:11.759959552  Train/Loss
1    99  1.133371 2024-07-19 10:09:12.169078272  Train/Loss
2   149  0.997314 2024-07-19 10:09:12.568946176  Train/Loss
3   199  1.021909 2024-07-19 10:09:12.959544576  Train/Loss
4   249  0.680294 2024-07-19 10:09:13.346163200  Train/Loss


In [6]:
concat_df = df

In [16]:
# concat_df["step"] > 5400  

array([   49,    99,   149, ..., 69272, 76969, 76970])

In [11]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_25273/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,76970.0,0.9447
Test/F1Score,76970.0,0.9369
Test/Loss,76970.0,0.1273
Test/Precision,76970.0,0.9475
Test/Recall,76970.0,0.9447


In [8]:
concat_df.to_csv("results_csvs/dfl_15_star.csv")